# Faza 3 uloha 4

In [1]:
import pandas as pd
import numpy as np

filename2 = "020/labor.csv"
labor = pd.read_csv(filename2, sep='\t', index_col=0)

labor.loc[labor['smoker'] == 'yes', 'smoker']='Y'
labor.loc[labor['smoker'] == 'no', 'smoker']='N'

labor['weight']=labor['weight'].apply(abs)
labor = labor.drop_duplicates()

indic = labor['indicator']
labor.drop(labels=['indicator'], axis=1,inplace = True)
labor.insert(0,'indicator',indic)

labor = labor.reset_index(drop = True)

labor=labor.drop(['name','relationship','ssn', 'smoker'],axis=1)
labor

,indicator,er-cv,leukocyty,weight,etytr,erytrocyty,trombocyty,hemoglobin,ast,alp,alt,hematokrit,hbver
0,1.0,43.09887,4.47526,65.57593,5.96219,6.20665,5.71427,5.06667,57.92930,57.60257,5.12934,7.11081,6.32708
1,1.0,32.51734,4.41339,58.47779,5.44840,5.58217,6.29619,6.90945,61.47846,83.38342,10.59365,8.52324,9.06429
2,1.0,54.39106,4.71119,37.58396,6.75478,6.36030,6.67129,4.08668,59.37033,31.60935,10.72644,8.41874,7.11435
3,1.0,45.79943,5.38079,88.06628,6.33099,5.45784,6.24957,5.57164,50.31551,75.29715,9.26557,4.69702,7.00724
4,0.0,48.81261,4.72469,44.59816,6.62153,6.36262,6.44608,6.49617,50.44424,84.90135,9.31196,6.31221,8.54503
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9883,1.0,57.11218,5.26323,51.27708,5.00976,4.91856,6.48513,3.03796,41.27456,13.47601,13.88191,5.86663,6.55392
9884,0.0,55.86133,8.38651,152.99318,9.27774,5.78542,4.73446,4.92447,44.92183,56.03309,8.23807,6.50680,7.18183
9885,1.0,20.08620,4.81549,89.14354,6.54037,5.12928,5.28566,7.07261,20.39526,75.88338,7.06386,5.16492,8.92604
9886,1.0,40.47220,5.28603,18.62037,6.22798,6.35608,6.37370,5.59105,18.04463,79.92407,12.06706,5.72674,8.84950


In [2]:
def my_fill_na(arg1,ind):
    labor.loc[labor['indicator'] == ind , arg1] = labor.loc[labor['indicator'] == ind , arg1].fillna(
    (labor.loc[labor['indicator'] == ind , arg1].mean()))
    
def drop_na(df):
    return df.copy().dropna()

def use_my_fill_na(atributes):
    for atribute in atributes:
        my_fill_na(atribute, 0)
        my_fill_na(atribute, 1)

In [3]:
# MISSING VALUES
# labor = drop_na(labor)
use_my_fill_na(labor.columns)

In [4]:
import random
def detect_outlier(atr):
    lower = atr.quantile(0.25) - 1.5
    upper = atr.quantile(0.75) + 1.5
    
    return atr[(atr > upper) | (atr < lower)]

def detect_upper_lower_outliers(atr):
    lower = atr.quantile(0.25) - 1.5
    upper = atr.quantile(0.75) + 1.5
    
    return atr[(atr > upper)], atr[(atr < lower)]

# Tato funkcia najde vrchnych a spodnych outlierov a vyplni ich nahodnou hodnotou medzi minimom bez outlierov a 10 spodnych % pripadne maximom bez outlierov a 10 vrchnych %
def fill_outliers(df, atr):
    df.loc[detect_upper_lower_outliers(df[atr])[0].index, [atr]] = random.uniform(df.drop(detect_upper_lower_outliers(df[atr])[0].index)[atr].quantile(0.90), df.drop(detect_upper_lower_outliers(df[atr])[0].index)[atr].max())
    df.loc[detect_upper_lower_outliers(df[atr])[1].index, [atr]] = random.uniform(df.drop(detect_upper_lower_outliers(df[atr])[1].index)[atr].min(), df.drop(detect_upper_lower_outliers(df[atr])[1].index)[atr].quantile(0.10))
    df[atr]
    
def handle_outlier(df, atributes, handle_type='fill'):
    result = df.copy()
    if handle_type == 'fill':
        for atribute in atributes:
            fill_outliers(result, atribute)
    elif handle_type == 'drop':
        for atribute in atributes:
            result.drop(detect_outlier(result[atribute]).index, inplace=True)
    else:
        print("incorrect handle_type")
    return result

In [5]:
# OUTLINERS
labor = handle_outlier(labor, labor.columns, 'fill')
# labor = handle_outlier(labor, labor.columns, 'drop')

In [6]:
# PREPROCESSING
split_by = 0.75
labor_train = labor.sample(frac=split_by,random_state=200)
labor_train.sort_index()

labor_test = labor
labor_test = pd.merge(labor_test,labor_train,indicator = True, how = 'outer').query('_merge=="left_only"').drop('_merge', axis = 1)

In [7]:
# TRANSFORMATION
atributes = labor_train.columns
def my_fit(df, scaler, atributes):
    return scaler.fit_transform(df.loc[:, atributes])

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaled = scaler.fit_transform(labor_train.loc[:, atributes])

# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# scaled = scaler.fit_transform(labor_train.loc[:, atributes])

# from sklearn.preprocessing import RobustScaler
# scaler = RobustScaler()
# scaled = scaler.fit_transform(labor_train.loc[:, atributes])

# from sklearn.preprocessing import PowerTransformer
# power = PowerTransformer(method='yeo-johnson', standardize=True)
# trans = power.fit_transform(labor_train.loc[:, atributes])

# from sklearn.preprocessing import QuantileTransformer
# qt = QuantileTransformer(n_quantiles=10, random_state=0)
# labor_train = my_fit(labor_train, qt, atributes)

In [8]:
# VYBER ATRIBUTOV
k=7
labor_train =  pd.DataFrame(labor_train,columns = atributes)

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_regression

y = labor_train['indicator']
labor_train_proc = SelectKBest(mutual_info_regression, k=k).fit_transform(labor_train, y)

# Vráti array názvov stĺpcov
def get_names(w_names, wo_names):
    names = w_names.columns
    ret_val = []
    i = 0
    for x in wo_names[0]:
        j = 0
        for y in w_names.iloc[0]:
            if j<i:
                j+=1
                continue
            if x == y:
                ret_val.append(names[j])
                i = j
                break
            j+=1
    return (ret_val)

labor_train_proc = pd.DataFrame(labor_train_proc,columns = get_names(labor_train,labor_train_proc))

from numpy import arange

def get_order():
    y = labor_train_proc['indicator']
    for x in arange(2, k+1, 1):
        new_arr = SelectKBest(mutual_info_regression, k=x).fit_transform(labor_train_proc, y)
        print(get_names(labor_train, new_arr)[1:])
        
get_order()

['leukocyty']
['leukocyty', 'hematokrit']
['leukocyty', 'hemoglobin', 'hematokrit']
['leukocyty', 'trombocyty', 'hemoglobin', 'hematokrit']
['leukocyty', 'trombocyty', 'hemoglobin', 'ast', 'hematokrit']
['leukocyty', 'etytr', 'trombocyty', 'hemoglobin', 'ast', 'hematokrit']


In [9]:
# LEARNING 2
X_train = labor_train[['er-cv','leukocyty','trombocyty','hemoglobin','ast','hematokrit']]
y_train = labor_train[['indicator']]
X_test = labor_test[['er-cv','leukocyty','trombocyty','hemoglobin','ast','hematokrit']]
y_test = labor_test[['indicator']]

from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn import tree

def predict_tree(i):
    if (i == 0):
        clf = DecisionTreeClassifier()
    else:
        print("Depth = ",i)
        clf = DecisionTreeClassifier(max_depth = i)
    clf.fit(X_train,y_train)
    pred = clf.predict(X_test)
    acc = accuracy_score(y_test, pred)
    rec = recall_score(y_test, pred)
    prec= precision_score(y_test, pred)
    print("Accuracy = ",acc," Precision = ",prec," Recall = ",rec)
    return clf

predict_tree(8)

Depth =  8
Accuracy =  0.8628640776699029  Precision =  0.8781538461538462  Recall =  0.9100765306122449


DecisionTreeClassifier(max_depth=8)

In [10]:
# LEARNING 3
def predict_tree_MinSample(i,j):
    print("Depth = ",i," Min sample leaf = ",j)
    clf = DecisionTreeClassifier(max_depth = i,min_samples_leaf = j)
    clf.fit(X_train,y_train)
    pred = clf.predict(X_test)
    acc = accuracy_score(y_test, pred)
    rec = recall_score(y_test, pred)
    prec= precision_score(y_test, pred)
    print("Accuracy = ",acc," Precision = ",prec," Recall = ",rec)
    return clf

predict_tree_MinSample(8,51)

Depth =  8  Min sample leaf =  51
Accuracy =  0.8725728155339806  Precision =  0.8992989165073295  Recall =  0.8998724489795918


DecisionTreeClassifier(max_depth=8, min_samples_leaf=51)

## VYSLEDKY
Ako defoultne budem pouzivat rozlozenie: 
- Strategia riesenia chybajucich hodnot: my_fill_na
- Strategia riesenia outlinerov: fill_outliners
- Scaling resp Transforming: MinMaxScaler
- Vyber atributov: k=7
- Ensamble learning: predict_tree_MinSample

Postupne budeme v kazdej kategorii menit vybrane algoritmy a vysledky accuracy zapisovat

### Strategia riesenia chybajucich hodnot
- my_fill_na: 0.8697411003236246
- drop_na: 0.8694560669456067

### Strategia riesenia outlinerov
- fill: 0.8697411003236246
- drop: 0.8981481481481481

### Scaling resp Transforming
- MinMaxScaler: 0.8697411003236246
- StandartScaler: 0.8685275080906149
- RobustScaler: 0.8673139158576052
- PowerTransformer: 0.8689320388349514
- QuantileTranformer: 0.3656957928802589

### Vyber Atributov
- k=3: 0.8681229773462783
- k=5: 0.8697411003236246
- k=7: 0.8697411003236246

### Ensamble learning
- OneR: 0.7148058252427184 (src: F3-uloha1.ipynb)
- PredictTree: 0.8717637540453075
- PredictTreeMinMax: 0.8677184466019418

## ZHODNOTENIE
Najlepsiu accuracy sme dosiahli pri nasledujucich nastaveniach:
- Strategia riesenia chybajucich hodnot: my_fill_na
- Strategia riesenia outlinerov: **drop_outliners**
- Scaling resp Transforming: MinMaxScaler
- Vyber atributov: k=7
- Ensamble learning: predict_tree_MinSample